Este notebook esta basado en el [paper](http://web.archive.org/web/20060720040135/http://members.fortunecity.com/jpeschel/gillog1.htm) utilizado para crackear en el M4 Project.

In [28]:
from enigma.enigma import Enigma
from enigma.rotors.rotor_I import RotorI
from enigma.rotors.rotor_II import RotorII
from enigma.rotors.rotor_III import RotorIII
from enigma.rotors.rotor_IV import RotorIV
from enigma.rotors.rotor_V import RotorV
from enigma.reflectors.reflector_b import ReflectorB
from enigma.plugboard import Plugboard
from language_models.character_frequency_kld_language_model import CharacterFrequencyKLDLanguageModel
from language_models.character_frequency_ic_language_model import CharacterFrequencyICLanguageModel
from language_models.markov_chain_model import MarkovChainModel

import random
from collections import OrderedDict
import heapq
from itertools import permutations
from tqdm import tqdm
from tqdm.auto import trange
from multiprocessing import Pool
from functools import partial

# Orden de los rotores

Vamos a simular la selección de los rotores I, II, III, IV y V para la Enigma M3 como esta descripto en el paper. Vamos a utilizar siempre el reflector B.

In [2]:
POSSIBLE_ROTORS = {RotorI, RotorII, RotorIII, RotorIV, RotorV}
POSSIBLE_LETTERS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def config_to_string(rotors_classes, offsets, plugboard_tuples):
    string = ""
    for rot, off in zip(rotors_classes, offsets):
        string += rot.__name__+str(off)
    for l1, l2 in plugboard_tuples:
        string += l1+l2
    return string

def get_random_config(plugboard_size: int):
    rotors_to_use = random.sample(POSSIBLE_ROTORS, 3)
    offsets = [random.randint(0,25) for _ in range(3)]
    letter_sample = random.sample(list(POSSIBLE_LETTERS), plugboard_size*2)
    plugboard_tuples = []
    for i in range(len(letter_sample)//2):
        plugboard_tuples.append((letter_sample[i*2], letter_sample[(i*2)+1]))
    plugboard_tuples = sorted(plugboard_tuples, key=lambda x: x[0])
    return rotors_to_use, offsets, plugboard_tuples

def transform_to_valid_chars(text: str):
    text = text.upper()
    return "".join([c for c in text if c in POSSIBLE_LETTERS])
        

In [3]:
config_to_string(*get_random_config(10))

'RotorV7RotorIV15RotorIII14GAHPJDLBNQOMSRWEYCZT'

In [4]:
with open('books/Alices_Adventures_in_Wonderland.txt', 'r') as book:
    book_train = transform_to_valid_chars(book.read())
with open('books/Pride_and_Prejudice.txt', 'r') as book:
    book_test = transform_to_valid_chars(book.read())

In [5]:
kld_char_freq = CharacterFrequencyKLDLanguageModel(book_train)
ic_char_freq = CharacterFrequencyICLanguageModel(book_train)
markov_chain1 = MarkovChainModel(book_train)

In [23]:
TEST_TEXT_LENGHT = 1000

rotors, offsets, plugboard_tuples =get_random_config(10)
random_config_key = config_to_string(rotors,offsets,plugboard_tuples)
random_test_pos = random.randint(0,len(book_test)- TEST_TEXT_LENGHT)
random_text_test = book_test[random_test_pos:random_test_pos+TEST_TEXT_LENGHT]
rotors_to_use = [rot_c(offset=off) for rot_c, off in zip(rotors, offsets)]
random_engima = Enigma(reflector=ReflectorB(),
                      plugboard=Plugboard(plugboard_tuples), rotors=rotors_to_use)
cyphertext = random_engima.encrypt(random_text_test)

### Explorar todos los rotores

In [ ]:
scores = {}

def compute_scores(rot1, rot2, rot3, offsets):
    off1 = offsets % 26
    off2 = (offsets//26) % 26
    off3 = (offsets//(26**2)) % 26
    enigma = Enigma(reflector=ReflectorB(),
                      plugboard=Plugboard(), 
                        rotors=[rot1(offset=off1), rot2(offset=off2), rot3(offset=off3)])
    decrypted = enigma.decrypt(cyphertext)
    config = config_to_string([rot1, rot2, rot3], [off1,off2,off3], [])
    return config, {'KLD': kld_char_freq.fitness(decrypted), 'IC': ic_char_freq.fitness(decrypted), 'Markov': markov_chain1.fitness(decrypted)}

for rot1, rot2, rot3 in permutations(POSSIBLE_ROTORS, 3):
    with Pool(8) as p:
        partial_compute = partial(compute_scores, rot1, rot2, rot3)
        r = list(tqdm(p.imap(partial_compute, range(26**3)), total=26**3))
    for c, s in r:
        scores[c] = s
        
        

 48%|████▊     | 8361/17576 [00:13<00:15, 605.48it/s]